In [ ]:
import os
import re
import json
from matplotlib import dates
import matplotlib.pyplot as plt
# Choose Style
# 'classic', 'ggplot', 'seaborn', 'Solarize_Light2', 'bmh', 'fivethirtyeight', 'dark_background', 'grayscale',  
plt.style.use('default')

from utils import *
from pprint import pprint

# Main

## Visualized

In [ ]:
# Get names of files
base_dir = '/home/wmnlab/E/database'
date = '2024-05-08'
exp = 'Modem_Action_Test_v6'
trace = '#05'

date_dir = os.path.join(base_dir, date)
d = os.path.join(date_dir, exp)
devs = sorted([a for a in os.listdir(d) if ('qc' in a or 'sm' in a)])
dev1, dev2 = devs[0], devs[1]
data_dir1 = os.path.join(d, dev1, trace, 'data')
data_dir2 = os.path.join(d, dev2, trace, 'data')
rrc_file1 = [os.path.join(data_dir1, f) for f in os.listdir(data_dir1) if 'rrc' in f][0]
rrc_file2 = [os.path.join(data_dir2, f) for f in os.listdir(data_dir2) if 'rrc' in f][0]
ml1_file1 = [os.path.join(data_dir1, f) for f in os.listdir(data_dir1) if ('ml1' in f and not 'nr_ml1' in f)][0]
ml1_file2 = [os.path.join(data_dir2, f) for f in os.listdir(data_dir2) if ('ml1' in f and not 'nr_ml1' in f)][0]
nr_ml1_file1 = [os.path.join(data_dir1, f) for f in os.listdir(data_dir1) if 'nr_ml1' in f][0]
nr_ml1_file2 = [os.path.join(data_dir2, f) for f in os.listdir(data_dir2) if 'nr_ml1' in f][0]
rrc_file1, rrc_file2, ml1_file1, ml1_file2, nr_ml1_file1, nr_ml1_file2

In [ ]:
HOs = parse_mi_ho(rrc_file1)
MRs = MeasureReport(rrc_file1)
MRs = correct_MR_with_HO(MRs, HOs)
mappings = map_MR_HO(MRs, HOs)
ordered_HOs1 = print_trans(HOs, mappings=mappings)

In [ ]:
HOs = parse_mi_ho(rrc_file2)
MRs = MeasureReport(rrc_file2)
MRs = correct_MR_with_HO(MRs, HOs)
mappings = map_MR_HO(MRs, HOs)
ordered_HOs2 = print_trans(HOs, mappings=mappings)

In [ ]:
dl_loss_excl_file1 = os.path.join(data_dir1, 'udp_dnlk_loss_latency.csv')
dl_loss_excl_file2 = os.path.join(data_dir2, 'udp_dnlk_loss_latency.csv')
dl_loss1, dl_excl1 = loss_excl_cause(dl_loss_excl_file1, rrc_file1)
dl_loss2, dl_excl2 = loss_excl_cause(dl_loss_excl_file2, rrc_file2)
dl_loss, dl_excl = loss_excl_cause_dual(dl_loss_excl_file1, dl_loss_excl_file2, rrc_file1, rrc_file2)

In [ ]:
ul_loss_excl_file1 = os.path.join(data_dir1, 'udp_uplk_loss_latency.csv')
ul_loss_excl_file2 = os.path.join(data_dir2, 'udp_uplk_loss_latency.csv')
ul_loss1, ul_excl1 = loss_excl_cause(ul_loss_excl_file1, rrc_file1)
ul_loss2, ul_excl2 = loss_excl_cause(ul_loss_excl_file2, rrc_file2)
ul_loss, ul_excl = loss_excl_cause_dual(ul_loss_excl_file1, ul_loss_excl_file2, rrc_file1, rrc_file2)

In [ ]:
len(dl_loss), len(dl_excl), len(ul_loss), len(ul_excl)

In [ ]:
# Read time sync file
time_sync_file = [os.path.join(date_dir, x) for x in os.listdir(date_dir) if x.endswith('.json') and 'time_sync' in x][0]
print(time_sync_file)
with open(time_sync_file, 'r') as f:
    time_off_dict = json.load(f)

pattern = r'(\d{4}-\d+-\d+_\d{2}-\d{2}-\d{2})'
match = re.search(pattern, rrc_file1)
if match:
    datetime_str = match.group(1)
    filetime = pd.to_datetime(datetime_str, format='%Y-%m-%d_%H-%M-%S')
else:
    print("File Name Error!"); raise

measure_times = [pd.to_datetime(k, format='%Y-%m-%d %H:%M:%S.%f') for k in time_off_dict.keys()]
time_diffs = [abs(t-filetime) for t in measure_times]
ind, value = min(enumerate(time_diffs), key=lambda x: x[1])
print('rrc file time:', filetime)
delta = dt.timedelta(seconds=list(time_off_dict.values())[ind])
print('Sync Time:', list(time_off_dict.keys())[ind], '; delta',list(time_off_dict.values())[ind])

In [ ]:
# Read TimeSync csv
dev_dir = os.path.join(date_dir, 'others/TimeSync', dev1)
files = os.listdir(dev_dir)
value = max(files, key = lambda x: find_longest_common_substring_length(x, rrc_file1))
ind = files.index(value)
TimeSync_file1 = os.path.join(dev_dir, files[ind])

dev_dir = os.path.join(date_dir, 'others/TimeSync', dev2)
files = os.listdir(dev_dir)
value = max(files, key = lambda x: find_longest_common_substring_length(x, rrc_file2))
ind = files.index(value)
TimeSync_file2 = os.path.join(dev_dir, files[ind])

print(rrc_file1, TimeSync_file1, rrc_file2, TimeSync_file2, sep='\n')
# names=['cell time', 'device time']
TS_df1 = pd.read_csv(TimeSync_file1, names=['cell time', 'device time'])
TS_df1['cell time'] = pd.to_datetime(TS_df1['cell time']) + dt.timedelta(hours=8)
TS_df1['device time'] = pd.to_datetime(TS_df1['device time'])

TS_df2 = pd.read_csv(TimeSync_file2, names=['cell time', 'device time'])
TS_df2['cell time'] = pd.to_datetime(TS_df2['cell time']) + dt.timedelta(hours=8)
TS_df2['device time'] = pd.to_datetime(TS_df2['device time'])

In [ ]:
# Align ordered_HOs HO time to server time
ordered_HOs1_ = ordered_HOs1 # store previous value
ordered_HOs2_ = ordered_HOs2 # store previous value
ordered_HOs1 = ho_time_to_server_time(ordered_HOs1, TS_df1, delta)
ordered_HOs2 = ho_time_to_server_time(ordered_HOs2, TS_df2, delta)

In [ ]:
start_time = dt.datetime(2024, 5, 8, 16, 45, 50, 000000)
end_time = dt.datetime(2024, 5, 8, 16, 45, 58, 000000)

time_range = (start_time, end_time)

R1_PCell, R1_SCell1, R1_SCell2, R1_SCell3, R1_Cells = LTE_signal_strength(ml1_file1, time_range)
R2_PCell, R2_SCell1, R2_SCell2, R2_SCell3, R2_Cells = LTE_signal_strength(ml1_file2, time_range)

R1_PSCell, R1_NR_Cells = NR_signal_strength(nr_ml1_file1, time_range)
R2_PSCell, R2_NR_Cells = NR_signal_strength(nr_ml1_file2, time_range)

In [ ]:
# Align SS time to server time
R1_PCell_, R1_SCell1_, R1_SCell2_, R1_SCell3_, R1_Cells_ = R1_PCell, R1_SCell1, R1_SCell2, R1_SCell3, R1_Cells
[R1_PCell, R1_SCell1, R1_SCell2, R1_SCell3] = [ss_time_to_server_time_way2(cell, TS_df1, delta) for cell in [R1_PCell, R1_SCell1, R1_SCell2, R1_SCell3]]
R1_PCell = add_nan(R1_PCell)
R1_Cells = {k: ss_time_to_server_time(cell, TS_df1, delta) for k, cell in R1_Cells.items()}

R2_PCell_, R2_SCell1_, R2_SCell2_, R2_SCell3_, R2_Cells_ = R2_PCell, R2_SCell1, R2_SCell2, R2_SCell3, R2_Cells
[R2_PCell, R2_SCell1, R2_SCell2, R2_SCell3] = [ss_time_to_server_time_way2(cell, TS_df2, delta) for cell in [R2_PCell, R2_SCell1, R2_SCell2, R2_SCell3]]
R2_Cells = {k: ss_time_to_server_time(cell, TS_df2, delta) for k, cell in R2_Cells.items()}

R1_PSCell_, R1_NR_Cells_ = R1_PSCell, R1_NR_Cells
R1_PSCell = ss_time_to_server_time(R1_PSCell_, TS_df1, delta)
R1_NR_Cells ={k: ss_time_to_server_time(cell, TS_df1, delta) for k, cell in R1_NR_Cells.items()}

R2_PSCell_, R2_NR_Cells_ = R2_PSCell, R2_NR_Cells
R2_PSCell = ss_time_to_server_time_way2(R2_PSCell_, TS_df2, delta)
R2_PSCell = add_nan(R2_PSCell)
R2_NR_Cells ={k: ss_time_to_server_time(cell, TS_df2, delta) for k, cell in R2_NR_Cells.items()}

In [ ]:
# Calculate Latency
R1_dl_pkgs = accumulate_packet(dl_loss_excl_file1,time_range)
R1_dl_pkgs = add_nan_pkg(R1_dl_pkgs)
R2_dl_pkgs = accumulate_packet(dl_loss_excl_file2,time_range)
R2_dl_pkgs = add_nan_pkg(R2_dl_pkgs)

In [ ]:
T1, Type1, Trans1, Ev1 = get_info(ordered_HOs1, time_range)
T2, Type2, Trans2, Ev2 = get_info(ordered_HOs2, time_range)

fig, axes = plt.subplots(4,1,figsize=(10, 6), sharex=True) 
HO_SCA1 = axes[0].scatter(T1, [1] * len(T1), marker='*', color='b', s=100, zorder=3, label='R1 HO')
HO_SCA2 = axes[0].scatter(T2, [1] * len(T2), marker='o', color='r', s=100, zorder=3, label='R2 HO')

axes[0].axhline(y=1, color='gray', linestyle='--', linewidth=1)

for event, type, trans in zip(T1, Type1, Trans1):
    if type in ['LTE HO', 'MN HO']:
        # continue
        trans = trans.split(' | ')[0]
        text = type + '\n' + get_pci(trans, 'eNB HO')
    elif type in ['RLF II']:
        trans = trans.split(' | ')[0]
        text = type + '\n' + get_pci(trans, 'eNB HO')
    elif type == 'MN HO to eNB':
        trans = trans.split(' | ')[0]
        text = 'MN HO\nto eNB'+ '\n' + get_pci(trans, 'eNB HO')
    elif type in ['SN HO']:
        # continue
        trans = trans.split(' | ')[1]
        text = type + '\n' + get_pci(trans, 'gNB HO')
    elif type == 'SN setup':
        continue
        trans = trans.split(' | ')[1]
        text = type + '\n' + get_pci(trans, 'gNB setup')
    elif type == 'Conn Req':
        text = type + '\n' + get_pci(trans, 'eNB HO')
    elif type == 'SCG RLF':
        continue
        text = 'SCG\nRLF'
    elif type == 'SN Rel':
        # continue
        text = 'SN\nRel' + '\n' + get_pci(trans, 'gNB rel')
    
    axes[0].text(event, 1.01, text, ha='center', va='bottom', fontsize=8, color='b', zorder=3)

shift = 0.01
for event, type, trans in zip(T2, Type2, Trans2):
    if type in ['LTE HO', 'MN HO']:
        # continue
        trans = trans.split(' | ')[0]
        text = type + '\n' + get_pci(trans, 'eNB HO')
    elif type in ['RLF II']:
        trans = trans.split(' | ')[0]
        text = type + '\n' + get_pci(trans, 'eNB HO')
    elif type == 'RLF III':
        text = type
    elif type == 'MN HO to eNB':
        # continue
        trans = trans.split(' | ')[0]
        text = 'MN HO\nto eNB'+ '\n' + get_pci(trans, 'eNB HO')
    elif type in ['SN HO']:
        # continue
        trans = trans.split(' | ')[1]
        text = type + '\n' + get_pci(trans, 'gNB HO')
    elif type == 'SN setup':
        continue
        trans = trans.split(' | ')[1]
        text = type + '\n' + get_pci(trans, 'gNB setup')
    elif type == 'Conn Req':
        # continue
        text = type + '\n' + get_pci(trans, 'eNB HO')
    elif type == 'SCG RLF':
        continue
        text = 'SCG\nRLF'
    elif type == 'SN Rel':
        continue
        text = 'SN\nRel' + '\n' + get_pci(trans, 'gNB rel')
        
    # plt.text(event, 0.99, text, ha='center', va='top', fontsize=8, color='r', zorder=3)
    axes[0].text(event, 0.99-abs(shift)/2-shift/2, text, ha='center', va='top', fontsize=8, color='r', zorder=3)
    shift*=-1
    
# plt.title('Events of Radio1 & Radio2')

axes[0].set_yticks([])
axes[0].set_xlim((start_time, end_time))

x_range = axes[0].get_xlim()
x_range = [dates.num2date(n).replace(tzinfo=None) for n in x_range]
y_range = axes[0].get_ylim()

# axes[0].text((x_range[1] - x_range[0])*1/2+x_range[0], (y_range[1] - y_range[0])*4.2/5+y_range[0],  
#         '5G Radio1', ha='center', va='center', fontsize=12, color='b', zorder=5)
# axes[0].text((x_range[1] - x_range[0])*1/2+x_range[0], (y_range[1] - y_range[0])*0.8/5+y_range[0], 
#         '5G Radio2', ha='center', va='center', fontsize=12, color='r', zorder=5)

L1 = [pkg.timestamp for pkg in dl_loss1]
L2 = [pkg.timestamp for pkg in dl_loss2]
DL_LOSS1 = axes[0].vlines(x=L1, ymin=1, ymax=y_range[1], alpha=1, zorder=2, colors='lightblue', label='R1 DL Loss')
DL_LOSS2 = axes[0].vlines(x=L2, ymin= y_range[0], ymax=1, alpha=1, zorder=2, colors='lightpink', label='R2 DL Loss')

L3 = [pkg.timestamp for pkg in dl_excl1]
L4 = [pkg.timestamp for pkg in dl_excl2]
DL_EXCL1 = axes[0].vlines(x=L3, ymin=1, ymax=y_range[1], alpha=1, zorder=1, colors='lightgreen', label='R1 DL EXCL')
DL_EXCL2 = axes[0].vlines(x=L4, ymin=y_range[0], ymax=1, alpha=1, zorder=1, colors='gold', label='R2 DL EXCL')

# plot latency
R1_Lat = axes[1].plot([pkg.Timestamp for pkg in R1_dl_pkgs], [pkg.latency for pkg in R1_dl_pkgs], 
            zorder=1, label='R1 Latency', linestyle='-')[0]
R2_Lat = axes[1].plot([pkg.Timestamp for pkg in R2_dl_pkgs], [pkg.latency for pkg in R2_dl_pkgs], 
            zorder=1, label='R2 Latency', linestyle='-')[0]
axes[1].set_ylim([0,0.5])
axes[1].set_ylabel('Latency\n(sec)')

# Add Signal Strength
R1_PCell_RSRP_PLOT = axes[2].plot([ss.Timestamp for ss in R1_PCell], [ss.RSRP for ss in R1_PCell], 
                            zorder=1, label='R1 RSRP', linestyle='--', marker='.')[0]
R2_PCell_RSRP_PLOT = axes[2].plot([ss.Timestamp for ss in R2_PCell], [ss.RSRP for ss in R2_PCell], 
                            zorder=1, label='R2 RSRP', linestyle='--', marker='.')[0]

R1_PSCell_RSRP_PLOT = axes[2].plot([ss.Timestamp for ss in R1_PSCell], [ss.RSRP for ss in R1_PSCell], 
                            zorder=1, label='R1 NR RSRP', linestyle='--', marker='.')[0]
R2_PSCell_RSRP_PLOT = axes[2].plot([ss.Timestamp for ss in R2_PSCell], [ss.RSRP for ss in R2_PSCell], 
                            zorder=1, label='R2 NR RSRP', linestyle='--', marker='.')[0]
axes[2].set_ylabel('RSRP\n(dBm)')

R1_PCell_RSRQ_PLOT = axes[3].plot([ss.Timestamp for ss in R1_PCell], [ss.RSRQ for ss in R1_PCell], 
                            zorder=1, label='R1 RSRQ', linestyle='--', marker='.')[0]
R2_PCell_RSRQ_PLOT = axes[3].plot([ss.Timestamp for ss in R2_PCell], [ss.RSRQ for ss in R2_PCell], 
                            zorder=1, label='R2 RSRQ', linestyle='--', marker='.')[0]

R1_PSCell_RSRQ_PLOT = axes[3].plot([ss.Timestamp for ss in R1_PSCell], [ss.RSRQ for ss in R1_PSCell], 
                            zorder=1, label='R1 NR RSRQ', linestyle='--', marker='.')[0]
R2_PSCell_RSRQ_PLOT = axes[3].plot([ss.Timestamp for ss in R2_PSCell], [ss.RSRQ for ss in R2_PSCell], 
                            zorder=1, label='R2 NR RSRQ', linestyle='--', marker='.')[0]
axes[3].set_ylabel('RSRQ\n(dB)')

# Legend
# ax.legend(handles = [LOSS1, LOSS2], loc='upper right', bbox_to_anchor=(1.25, 1))
axes[0].legend(handles = [DL_LOSS1, DL_LOSS2, HO_SCA1, HO_SCA2], loc='upper right', bbox_to_anchor=(1.20, 1.1))
axes[1].legend(loc='upper right',  bbox_to_anchor=(1.20, 1.1))
axes[2].legend(loc='upper right',  bbox_to_anchor=(1.20, 1.1))
axes[3].legend(loc='upper right',  bbox_to_anchor=(1.20, 1.1))

axes[-1].set_xlabel('Time')
# axes[0].get_shared_x_axes().join(axes[0], axes[1])


myFmt = dates.DateFormatter('%H:%M:%S') # fmt %H:%M:%S.%f
axes[-1].xaxis.set_major_formatter(myFmt)
fig.autofmt_xdate()

plt.show()


In [ ]:
T1, Type1, Trans1, Ev1 = get_info(ordered_HOs1, time_range)
T2, Type2, Trans2, Ev2 = get_info(ordered_HOs2, time_range)

fig, ax = plt.subplots() 
HO_SCA1 = ax.scatter(T1, [1] * len(T1), marker='*', color='b', s=100, zorder=3, label='R1 HO')
HO_SCA2 = ax.scatter(T2, [1] * len(T2), marker='o', color='r', s=100, zorder=3, label='R2 HO')

ax.axhline(y=1, color='gray', linestyle='--', linewidth=1)

for event, type, trans in zip(T1, Type1, Trans1):
    if type in ['LTE HO', 'MN HO']:
        # continue
        trans = trans.split(' | ')[0]
        text = type + '\n' + get_pci(trans, 'eNB HO')
    elif type in ['RLF II']:
        trans = trans.split(' | ')[0]
        text = type + '\n' + get_pci(trans, 'eNB HO')
    elif type == 'MN HO to eNB':
        trans = trans.split(' | ')[0]
        text = 'MN HO\nto eNB'+ '\n' + get_pci(trans, 'eNB HO')
    elif type in ['SN HO']:
        # continue
        trans = trans.split(' | ')[1]
        text = type + '\n' + get_pci(trans, 'gNB HO')
    elif type == 'SN setup':
        continue
        trans = trans.split(' | ')[1]
        text = type + '\n' + get_pci(trans, 'gNB setup')
    elif type == 'Conn Req':
        text = type + '\n' + get_pci(trans, 'eNB HO')
    elif type == 'SCG RLF':
        continue
        text = 'SCG\nRLF'
    elif type == 'SN Rel':
        # continue
        text = 'SN\nRel' + '\n' + get_pci(trans, 'gNB rel')
    
    ax.text(event, 1.01, text, ha='center', va='bottom', fontsize=8, color='b', zorder=3)

shift = 0.01
for event, type, trans in zip(T2, Type2, Trans2):
    if type in ['LTE HO', 'MN HO']:
        # continue
        trans = trans.split(' | ')[0]
        text = type + '\n' + get_pci(trans, 'eNB HO')
    elif type in ['RLF II']:
        trans = trans.split(' | ')[0]
        text = type + '\n' + get_pci(trans, 'eNB HO')
    elif type == 'RLF III':
        text = type
    elif type == 'MN HO to eNB':
        # continue
        trans = trans.split(' | ')[0]
        text = 'MN HO\nto eNB'+ '\n' + get_pci(trans, 'eNB HO')
    elif type in ['SN HO']:
        # continue
        trans = trans.split(' | ')[1]
        text = type + '\n' + get_pci(trans, 'gNB HO')
    elif type == 'SN setup':
        continue
        trans = trans.split(' | ')[1]
        text = type + '\n' + get_pci(trans, 'gNB setup')
    elif type == 'Conn Req':
        # continue
        text = type + '\n' + get_pci(trans, 'eNB HO')
    elif type == 'SCG RLF':
        continue
        text = 'SCG\nRLF'
    elif type == 'SN Rel':
        continue
        text = 'SN\nRel' + '\n' + get_pci(trans, 'gNB rel')
        
    # plt.text(event, 0.99, text, ha='center', va='top', fontsize=8, color='r', zorder=3)
    ax.text(event, 0.99-abs(shift)/2-shift/2, text, ha='center', va='top', fontsize=8, color='r', zorder=3)
    shift*=-1
    
# plt.title('Events of Radio1 & Radio2')

ax.set_yticks([])
ax.set_xlim((start_time, end_time))

x_range = ax.get_xlim()
x_range = [dates.num2date(n).replace(tzinfo=None) for n in x_range]
y_range = ax.get_ylim()

L1 = [pkg.timestamp for pkg in dl_loss1]
L2 = [pkg.timestamp for pkg in dl_loss2]
DL_LOSS1 = ax.vlines(x=L1, ymin=1, ymax=y_range[1], alpha=1, zorder=2, colors='lightblue', label='R1 DL Loss')
DL_LOSS2 = ax.vlines(x=L2, ymin= y_range[0], ymax=1, alpha=1, zorder=2, colors='lightpink', label='R2 DL Loss')


ax.set_xlabel('4 second Timeline')
# axes[0].get_shared_x_axes().join(axes[0], axes[1])


myFmt = dates.DateFormatter('%H:%M:%S') # fmt %H:%M:%S.%f
ax.xaxis.set_major_formatter(myFmt)
fig.autofmt_xdate()

plt.legend()
plt.show()
